## Scarping Liputan 6

In [21]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

In [22]:
threads_link = []
links = []
results = []
threads = []
keywords='anies'

In [23]:
def scrape_links(keywords):
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
    url = "https://www.liputan6.com/search?q=prabowo"
    
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(10)
    # driver.navigate(url)
    # time.sleep(10)
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Tunggu beberapa detik agar halaman dimuat
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            break
        last_height = new_height
        
    page_source = driver.page_source
    driver.quit()
    
    soup = BeautifulSoup(page_source, "html.parser")
    articles = soup.find_all('h4', {"articles--rows--item__title"})
    
    page_links = []
    for article in articles:
        link_element = article.find('a')
        if link_element is not None:
            link = link_element.get('href')
            if link is not None:
                page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {url}")
    return page_links

links = scrape_links(keywords)
print("Total Links:", len(links))


Scraped 0 links from page https://www.liputan6.com/search?q=prabowo
Total Links: 0


In [24]:
def scrape_url(url):
    try:
        headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Judul Berita
            title_elem = soup.find('h1', {"class": "read-page--header--title entry-title"})
            if title_elem:
                title_text = title_elem.text
            else:
                title_text = "Title not found"
            # # Author berita
            author_elem = soup.find('span', {"class": "read-page--header--author__name fn"})
            if author_elem:
                author_text =author_elem.text
            else:
                author_text = "Author not found"

            
            # # tanggal berita
            date_elem = soup.find('p', {"class": "read-page--header--author__datetime-wrapper"})
            if date_elem:
                date_text = date_elem.text.strip()
            else:
                date_text = "Date not found"
            #     # Category berita
            # category_elements = soup.find_all('span',{"itemprop": "name"})
            # if category_elements:
            #     category_text= category_elements[2].text
            # else:
            category_text = "Category not found"
            # ## Content Berita
            body_elem = soup.find('div', {"class": "article-content-body__item-content"})
            
            if body_elem:
                content_elem = body_elem.find_all('p')
                content_text = ""
                for p in content_elem:
                    content_text += p.text.strip() + "\n"
                
                if content_text.strip():
                    content_text=content_text
                else:
                    content_text ="Content not found"
            else:
              content_text ="Content not found"

            results.append({'title': title_text,
                            'author': author_text,
                            'date':date_text,
                            'category': category_text,
                            'content': content_text,
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [25]:

for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [26]:
df = pd.DataFrame(results)
print("Total Links:", len(results))
df.head(10)

Total Links: 0


""


In [27]:
filtered_df = df[df['content'].str.contains(keywords, case=False)|
                 df['title'].str.contains(keywords, case=False)]
print('hasil filter',len(filtered_df))
filtered_df.head(10)

KeyError: 'content'

Tempat Save File

In [ ]:
# current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# excel_file_name = f'liputan6_{current_datetime}.xlsx'
# filtered_df.to_excel(excel_file_name, index=False)

# print(f'Data has been saved to {excel_file_name}')